# Ethiopian Sign Language - MediaPipe Landmark Model
## Much better than image-based model!

This extracts hand landmarks (21 points x 3 coordinates = 63 features) from your images,
then trains a model on landmarks instead of raw pixels.

**Benefits:**
- Works regardless of background
- Works regardless of clothing
- Works in different lighting
- Much faster prediction
- Smaller model size

In [ ]:
# ============ STEP 1: INSTALL & MOUNT ============
!pip install mediapipe -q

from google.colab import drive
drive.mount('/content/drive')

# ===== CHANGE THESE PATHS! =====
DATA_DIR = "/content/drive/MyDrive/eth_frames/train"  # Your training images folder
MODEL_SAVE_PATH = "/content/drive/MyDrive/eth_landmark_model.keras"
LABELS_SAVE_PATH = "/content/drive/MyDrive/eth_landmark_labels.json"

print("Setup complete!")

In [ ]:
# ============ STEP 2: IMPORTS ============
import os
import json
import numpy as np
import cv2
import mediapipe as mp
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

print(f"TensorFlow: {tf.__version__}")
print(f"MediaPipe loaded!")
print(f"GPU: {tf.config.list_physical_devices('GPU')}")

In [ ]:
# ============ STEP 3: CHECK DATASET ============
print("Checking dataset...")

if os.path.exists(DATA_DIR):
    classes = sorted([d for d in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, d))])
    print(f"\n✅ Found {len(classes)} classes:")
    total_images = 0
    for i, cls in enumerate(classes):
        cls_path = os.path.join(DATA_DIR, cls)
        images = [f for f in os.listdir(cls_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        total_images += len(images)
        print(f"  {i}: {cls} - {len(images)} images")
    print(f"\nTotal images: {total_images}")
else:
    print(f"❌ Data folder not found: {DATA_DIR}")
    print("Please check your path!")

In [ ]:
# ============ STEP 4: EXTRACT LANDMARKS FROM IMAGES ============
def extract_hand_landmarks(image_path, hands):
    """
    Extract hand landmarks from an image.
    Returns 63 features (21 landmarks x 3 coordinates) or None if no hand detected.
    """
    image = cv2.imread(image_path)
    if image is None:
        return None
    
    # Convert BGR to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Process the image
    results = hands.process(image_rgb)
    
    if results.multi_hand_landmarks:
        # Get the first hand detected
        hand_landmarks = results.multi_hand_landmarks[0]
        
        # Extract 21 landmarks (x, y, z for each)
        landmarks = []
        for lm in hand_landmarks.landmark:
            landmarks.extend([lm.x, lm.y, lm.z])
        
        return np.array(landmarks)  # 63 features
    
    return None

# Extract landmarks from all images
print("Extracting hand landmarks from images...")
print("This may take a few minutes...\n")

X_data = []  # Landmarks
y_data = []  # Labels
failed_images = []

# Use MediaPipe Hands
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=1,
    min_detection_confidence=0.5
) as hands:
    
    for class_idx, class_name in enumerate(tqdm(classes, desc="Processing classes")):
        class_path = os.path.join(DATA_DIR, class_name)
        images = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        
        class_success = 0
        for img_name in images:
            img_path = os.path.join(class_path, img_name)
            landmarks = extract_hand_landmarks(img_path, hands)
            
            if landmarks is not None:
                X_data.append(landmarks)
                y_data.append(class_idx)
                class_success += 1
            else:
                failed_images.append(f"{class_name}/{img_name}")
        
        # print(f"  {class_name}: {class_success}/{len(images)} hands detected")

X_data = np.array(X_data)
y_data = np.array(y_data)

print(f"\n✅ Extraction complete!")
print(f"Total samples with hands detected: {len(X_data)}")
print(f"Failed to detect hands: {len(failed_images)}")
print(f"Feature shape: {X_data.shape}")

if len(failed_images) > 0 and len(failed_images) <= 20:
    print(f"\nFailed images: {failed_images}")

In [ ]:
# ============ STEP 5: VISUALIZE SAMPLE LANDMARKS ============
def visualize_landmarks(image_path, hands):
    """Show image with hand landmarks drawn"""
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                image_rgb, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    
    return image_rgb

# Show sample images with landmarks
print("Sample images with detected landmarks:")
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

with mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5) as hands:
    sample_idx = 0
    for i, class_name in enumerate(classes[:8]):
        class_path = os.path.join(DATA_DIR, class_name)
        images = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        if images:
            img_path = os.path.join(class_path, images[0])
            img_with_landmarks = visualize_landmarks(img_path, hands)
            
            ax = axes[sample_idx // 4, sample_idx % 4]
            ax.imshow(img_with_landmarks)
            ax.set_title(class_name)
            ax.axis('off')
            sample_idx += 1

plt.tight_layout()
plt.show()

In [ ]:
# ============ STEP 6: PREPARE DATA ============
from tensorflow.keras.utils import to_categorical

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_data, y_data, test_size=0.2, random_state=42, stratify=y_data
)

# Convert labels to one-hot encoding
num_classes = len(classes)
y_train_cat = to_categorical(y_train, num_classes)
y_test_cat = to_categorical(y_test, num_classes)

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")
print(f"Number of classes: {num_classes}")
print(f"Input shape: {X_train.shape[1]} features (21 landmarks x 3 coordinates)")

In [ ]:
# ============ STEP 7: DATA AUGMENTATION FOR LANDMARKS ============
def augment_landmarks(landmarks, num_augmented=5):
    """
    Augment landmark data by adding small random noise.
    This simulates slight variations in hand position.
    """
    augmented = [landmarks]
    
    for _ in range(num_augmented):
        # Add small random noise (simulates slight movement)
        noise = np.random.normal(0, 0.02, landmarks.shape)
        augmented.append(landmarks + noise)
        
        # Scale slightly (simulates distance from camera)
        scale = np.random.uniform(0.95, 1.05)
        augmented.append(landmarks * scale)
    
    return augmented

# Augment training data
print("Augmenting training data...")
X_train_aug = []
y_train_aug = []

for x, y in zip(X_train, y_train):
    augmented = augment_landmarks(x, num_augmented=3)
    X_train_aug.extend(augmented)
    y_train_aug.extend([y] * len(augmented))

X_train_aug = np.array(X_train_aug)
y_train_aug = to_categorical(np.array(y_train_aug), num_classes)

print(f"Original training samples: {len(X_train)}")
print(f"Augmented training samples: {len(X_train_aug)}")

In [ ]:
# ============ STEP 8: BUILD MODEL ============
print("Building landmark model...")

model = Sequential([
    Input(shape=(63,)),  # 21 landmarks x 3 coordinates
    
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    
    Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()
print(f"\nModel parameters: {model.count_params():,}")

In [ ]:
# ============ STEP 9: TRAIN MODEL ============
callbacks = [
    ModelCheckpoint(MODEL_SAVE_PATH, monitor='val_accuracy', save_best_only=True, verbose=1),
    EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6, verbose=1)
]

print("Training model...")
history = model.fit(
    X_train_aug, y_train_aug,
    validation_data=(X_test, y_test_cat),
    epochs=100,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

In [ ]:
# ============ STEP 10: PLOT TRAINING HISTORY ============
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(history.history['accuracy'], label='Train', linewidth=2)
ax1.plot(history.history['val_accuracy'], label='Validation', linewidth=2)
ax1.set_title('Model Accuracy', fontsize=14)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(history.history['loss'], label='Train', linewidth=2)
ax2.plot(history.history['val_loss'], label='Validation', linewidth=2)
ax2.set_title('Model Loss', fontsize=14)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(MODEL_SAVE_PATH.replace('.keras', '_history.png'), dpi=150)
plt.show()

In [ ]:
# ============ STEP 11: EVALUATE MODEL ============
print("Evaluating model...")

# Load best model
best_model = keras.models.load_model(MODEL_SAVE_PATH)

# Evaluate
test_loss, test_acc = best_model.evaluate(X_test, y_test_cat, verbose=0)
print(f"\n🎯 Test Accuracy: {test_acc*100:.2f}%")
print(f"📉 Test Loss: {test_loss:.4f}")

# Confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

y_pred = best_model.predict(X_test, verbose=0)
y_pred_classes = np.argmax(y_pred, axis=1)

print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes, target_names=classes))

In [ ]:
# ============ STEP 12: SAVE LABELS ============
# Save class labels
labels_dict = {str(i): classes[i] for i in range(len(classes))}

with open(LABELS_SAVE_PATH, 'w', encoding='utf-8') as f:
    json.dump(labels_dict, f, ensure_ascii=False, indent=2)

print(f"✅ Labels saved to: {LABELS_SAVE_PATH}")
print(f"\nLabel mapping:")
for idx, label in labels_dict.items():
    print(f"  {idx}: {label}")

# Print as Python list for copy-paste
print(f"\n📋 Copy this for your prediction code:")
print(f"ETH_LANDMARK_LABELS = {classes}")

In [ ]:
# ============ STEP 13: TEST PREDICTION ============
def predict_from_image(image_path, model, hands, classes):
    """Test prediction on a single image"""
    landmarks = extract_hand_landmarks(image_path, hands)
    
    if landmarks is None:
        return "No hand detected", 0
    
    # Predict
    landmarks = landmarks.reshape(1, -1)
    prediction = model.predict(landmarks, verbose=0)
    
    pred_idx = np.argmax(prediction[0])
    confidence = prediction[0][pred_idx] * 100
    
    return classes[pred_idx], confidence

# Test on random images
print("Testing predictions on sample images:\n")

with mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5) as hands:
    for class_name in classes[:5]:
        class_path = os.path.join(DATA_DIR, class_name)
        images = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        if images:
            img_path = os.path.join(class_path, images[-1])  # Use last image (not in training)
            pred_label, conf = predict_from_image(img_path, best_model, hands, classes)
            status = "✅" if pred_label == class_name else "❌"
            print(f"{status} True: {class_name} | Predicted: {pred_label} ({conf:.1f}%)")

In [ ]:
# ============ STEP 14: SUMMARY & DOWNLOAD ============
print("="*60)
print("✅ TRAINING COMPLETE!")
print("="*60)
print(f"\n📁 Model saved to: {MODEL_SAVE_PATH}")
print(f"📁 Labels saved to: {LABELS_SAVE_PATH}")
print(f"\n🎯 Test Accuracy: {test_acc*100:.2f}%")
print(f"📊 Number of classes: {num_classes}")
print(f"📊 Model size: ~{model.count_params() * 4 / 1024:.1f} KB")
print(f"\n⚡ This model uses hand landmarks (63 features)")
print(f"   instead of images (224x224x3 = 150,528 pixels)")
print(f"   = 2,389x smaller input!")
print(f"\n📥 Download these files and copy to your project:")
print(f"   1. {MODEL_SAVE_PATH}")
print(f"   2. {LABELS_SAVE_PATH}")

# Download
from google.colab import files
files.download(MODEL_SAVE_PATH)
files.download(LABELS_SAVE_PATH)